# Librerías a utilizar

In [ ]:
from funciones_auxiliares import configurar_pyplot
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Importar datos

Inicialmente, los datasets se ven como sigue:

In [ ]:
track_features = pd.read_csv('data/tf_mini.csv')
print('Dataset Track Features:')
print('- Filas:', track_features.shape[0])
print('- Columnas:', track_features.shape[1])
print('- Datos nulos:', track_features.isna().sum().sum())
display(track_features.head(3))

In [ ]:
sessions = pd.read_csv('data/log_mini.csv')
print('Dataset Sessions:')
print('- Filas:', sessions.shape[0])
print('- Columnas:', sessions.shape[1])
print('- Datos nulos:', sessions.isna().sum().sum())
display(sessions.head(3))

In [ ]:
spotify_songs = pd.read_csv('data/spotify_songs.csv')
print('Dataset Spotify Songs:')
print('- Filas:', spotify_songs.shape[0])
print('- Columnas:', spotify_songs.shape[1], spotify_songs.columns)
print('- Datos nulos:', spotify_songs.isna().sum().sum())
display(spotify_songs.head(3))

# Análisis

Cantidad de canciones por año de lanzamiento en `track_features`:

In [ ]:
configurar_pyplot(xlabel='Cantidad de canciones', ylabel='Año de lanzamiento')
plt.hist(x=track_features['release_year'], bins=20)

Cantidad de canciones por año de lanzamiento en `spotify_songs`:

In [ ]:
configurar_pyplot(xlabel='Cantidad de canciones', ylabel='Año de lanzamiento')
plt.hist(x=spotify_songs[spotify_songs['year'] > 0]['year'], bins=40)

Cantidad de sesiones con una determinada cantidad de reproducciones:

In [ ]:
cantidades = sessions
cantidades.loc[:, 'cuenta_1'] = 1
cantidades.loc[:, 'cuenta_2'] = 1
cantidades = cantidades.groupby('session_id').count().groupby('cuenta_1').count()['cuenta_2']
configurar_pyplot(xlabel='Cantidad de reproducciones', ylabel='Cantidad de sesiones')
plt.bar(x=np.arange(len(cantidades)), height=cantidades, tick_label=cantidades.index)

Cantidad de canciones con una determinada cantidad de reproducciones:

In [ ]:
cantidades = sessions
cantidades.loc[:, 'cuenta_1'] = 1
cantidades.loc[:, 'cuenta_2'] = 1
cantidades = cantidades.groupby('track_id_clean').count().groupby('cuenta_1').count()[['cuenta_2']]
N = 20
N_o_mas = cantidades.loc[N:, :].sum().values[0]
cantidades = cantidades.loc[:N-1, :]
cantidades.loc[f'{N}+', 'cuenta_2'] = N_o_mas
cantidades = cantidades['cuenta_2']
configurar_pyplot(xlabel='Cantidad de reproducciones', ylabel='Cantidad de canciones')
plt.bar(x=np.arange(len(cantidades)), height=cantidades, tick_label=cantidades.index)

Resumen del dataset `sessions`:

In [ ]:
resumen = pd.DataFrame()
resumen.loc['Usuarios', 'Valor'] = sessions['session_id'].nunique()
resumen.loc['Canciones', 'Valor'] = track_features['track_id'].nunique()
canciones_por_usuario = sessions.groupby('session_id').count().iloc[:, 0]
resumen.loc['Promedio de canciones por usuario', 'Valor'] = canciones_por_usuario.mean()
resumen.loc['Desviación estándar de canciones por usuario', 'Valor'] = canciones_por_usuario.std()
usuarios_por_cancion = sessions.groupby('track_id_clean').count().iloc[:, 0]
resumen.loc['Promedio de usuarios por canción', 'Valor'] = usuarios_por_cancion.mean()
resumen.loc['Desviación estándar de usuarios por canción', 'Valor'] = usuarios_por_cancion.std()
resumen.loc['Densidad del conjunto de datos', 'Valor'] = (
    sessions[['session_id', 'track_id_clean']].drop_duplicates().shape[0]
    /
    (sessions['session_id'].nunique() * sessions['track_id_clean'].nunique())
)
resumen